In [1]:
import os
import sys

# Get the absolute path of the current notebook's directory
current_dir = os.path.dirname(os.path.abspath(''))
print(f"Current directory: {current_dir}")

# Add the project root to Python path
if current_dir not in sys.path:
    sys.path.append(current_dir)
    
print(f"sys.path: {sys.path}")

# Now we can import from actions
from actions.form_location import ValidateLocationForm
from actions.base_form import BaseFormValidationAction
from actions.constants import QR_PROVINCE, QR_DISTRICT
from rasa_sdk import Tracker
from rasa_sdk.executor import CollectingDispatcher
from rasa_sdk.types import DomainDict
from unittest import TestCase

from unittest.mock import Mock, patch, ANY
import asyncio

print(f"Project root added to path: {current_dir}")

# Setup helper functions
def create_tracker(slots=None, latest_message=None):
    """Helper to create a tracker with specific slots and latest message."""
    if slots is None:
        slots = {}
    if latest_message is None:
        latest_message = {"text": "", "intent": {"name": ""}}
    
    mock_tracker = Mock(spec=Tracker)
    mock_tracker.slots = slots
    mock_tracker.latest_message = latest_message
    mock_tracker.get_slot = lambda x: slots.get(x)
    
    return mock_tracker

# Initialize form validator and common objects
form_validator = ValidateLocationForm()
domain = {
    "slots": {
        "user_municipality_temp": {"type": "text"},
        "user_municipality": {"type": "text"},
        "user_municipality_confirmed": {"type": "bool"},
        "user_village": {"type": "text"},
        "user_address_temp": {"type": "text"},
        "user_address_confirmed": {"type": "bool"},
        "user_address": {"type": "text"},
    }
}

print("Setup complete!")
print(QR_PROVINCE, QR_DISTRICT)

Current directory: /home/ubuntu/nepal_chatbot
sys.path: ['/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/ubuntu/nepal_chatbot/rasa-env/lib/python3.8/site-packages', '/home/ubuntu/nepal_chatbot']


2025-02-17 11:55:49.409456: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-17 11:55:49.411759: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-17 11:55:49.462291: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-17 11:55:49.463531: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-17 11:55:50.197493: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

PostgreSQL environment variables not found. Using SQLite
OpenAI key is loaded
Project root added to path: /home/ubuntu/nepal_chatbot
Setup complete!
KOSHI Jhapa


In [2]:
# Test municipality validation
async def test_municipality():
    dispatcher = CollectingDispatcher()
    tracker = create_tracker(
        slots={"requested_slot": "user_municipality_temp"},
        latest_message={"text": "Kathmandu", "intent": {"name": ""}}
    )
    
    result = await form_validator.validate_user_municipality_temp(
        "Kathmandu",
        dispatcher,
        tracker,
        domain
    )
    
    print("Test result:", result)
    print("Dispatcher messages:", dispatcher.messages)

# Run the test
await test_municipality()

######## FORM: Validating municipality_temp ######
Received value: Kathmandu
######## LocationValidator: Preprocessed text: kathmandu
######## LocationValidator: QR
######## LocationValidator: Score: 90.0
######## LocationValidator: Score: 100.0
######## LocationValidator: Municipality names: ['arjundhara', 'barhadashi', 'bhadrapur', 'birtamod', 'buddha shanti', 'damak', 'gauradaha', 'gaurigunj', 'haldibari', 'jhapa', 'kachanakawal', 'kamal', 'kankai', 'mechinagar', 'sitasatakshi']
######## LocationValidator: Result: {'province': 'koshi province', 'district': 'jhapa', 'error': 'Could not determine municipality in jhapa.'}
Validated municipality: None
Test result: {'user_municipality_temp': None, 'user_municipality': None, 'user_municipality_confirmed': None}
Dispatcher messages: [{'text': 'Please enter your municipality name in Jhapa, KOSHI', 'buttons': [], 'elements': [], 'custom': {}, 'template': None, 'response': None, 'image': None, 'attachment': None}]


In [3]:
# Second cell - Test class
class TestValidateLocationForm(TestCase):
    """Test cases for ValidateLocationForm class."""

    def setUp(self):
        """Set up test cases."""
        self.form_validator = ValidateLocationForm()
        self.domain = {
            "slots": {
                "user_municipality_temp": {"type": "text"},
                "user_municipality": {"type": "text"},
                "user_municipality_confirmed": {"type": "bool"},
                "user_village": {"type": "text"},
                "user_address_temp": {"type": "text"},
                "user_address_confirmed": {"type": "bool"},
                "user_address": {"type": "text"},
            }
        }

    def create_tracker(self, slots=None, latest_message=None):
        """Helper to create a tracker with specific slots and latest message."""
        if slots is None:
            slots = {}
        if latest_message is None:
            latest_message = {"text": "", "intent": {"name": ""}}
        
        mock_tracker = Mock(spec=Tracker)
        mock_tracker.slots = slots
        mock_tracker.latest_message = latest_message
        mock_tracker.get_slot = lambda x: slots.get(x)
        
        return mock_tracker

print("Test class defined!")

Test class defined!


In [4]:
# Municipality validation tests
async def test_municipality_validation():
    """Test municipality validation with different inputs."""
    test = TestValidateLocationForm()
    test.setUp()
    
    test_cases = [
        # Exact matches
        {
            "name": "Valid municipality - Arjundhara",
            "input": "Arjundhara Municipality",
            "slots": {"requested_slot": "user_municipality_temp"},
            "expected_result": {"user_municipality_temp": "Arjundhara"}
        },
        {
            "name": "Valid municipality - Kamal Rural Municipality",
            "input": "Kamal Rural Municipality",
            "slots": {"requested_slot": "user_municipality_temp"},
            "expected_result": {"user_municipality_temp": "Kamal"}
        },
        
        # Fuzzy matches - Single word
        {
            "name": "Fuzzy match - arjundhara lowercase",
            "input": "arjundhara",
            "slots": {"requested_slot": "user_municipality_temp"},
            "expected_result": {"user_municipality_temp": "Arjundhara"}
        },
        {
            "name": "Fuzzy match - arjundara typo",
            "input": "arjundara",
            "slots": {"requested_slot": "user_municipality_temp"},
            "expected_result": {"user_municipality_temp": "Arjundhara"}
        },
        
        # Fuzzy matches - Multi word
        {
            "name": "Fuzzy match - Kamal only",
            "input": "kamal",
            "slots": {"requested_slot": "user_municipality_temp"},
            "expected_result": {"user_municipality_temp": "Kamal"}
        },
        {
            "name": "Fuzzy match - Kamal Gaun",
            "input": "Kamal Gaun",
            "slots": {"requested_slot": "user_municipality_temp"},
            "expected_result": {"user_municipality_temp": "Kamal"}
        },
        {
            "name": "Fuzzy match - kamal gaun palika",
            "input": "kamal gaun palika",
            "slots": {"requested_slot": "user_municipality_temp"},
            "expected_result": {"user_municipality_temp": "Kamal"}
        },
        {
            "name": "Fuzzy match - kml gau",
            "input": "kml gau",
            "slots": {"requested_slot": "user_municipality_temp"},
            "expected_result": {"user_municipality_temp": "Kamal"}
        },
        
        # # Nepali script
        # {
        #     "name": "Nepali input - कमल",
        #     "input": "कमल",
        #     "slots": {"requested_slot": "user_municipality_temp"},
        #     "expected_result": {"user_municipality_temp": "Kamal Rural Municipality"}
        # },
        # {
        #     "name": "Nepali input - कमल गाउँपालिका",
        #     "input": "कमल गाउँपालिका",
        #     "slots": {"requested_slot": "user_municipality_temp"},
        #     "expected_result": {"user_municipality_temp": "Kamal Rural Municipality"}
        # },
        
        # Skip and Invalid cases
        {
            "name": "Skip request",
            "input": "skip",
            "slots": {"requested_slot": "user_municipality_temp"},
            "latest_message": {"text": "skip", "intent": {"name": "skip"}},
            "expected_result": {
                "user_municipality_temp": "Skipped",
                "user_municipality_confirmed": True,
                "provide_additional_location": False,
                "user_municipality": "Skipped"
            }
        },
        {
            "name": "Invalid municipality",
            "input": "NonExistent City",
            "slots": {"requested_slot": "user_municipality_temp"},
            "expected_result": {"user_municipality_temp": None}
        }
    ]
    
    for case in test_cases:
        print(f"\n{'='*50}")
        print(f"Testing: {case['name']}")
        print(f"{'='*50}")
        print(f"\nInput: {case['input']}")
        print(f"Expected result: {case['expected_result']}")
        
        try:
            latest_message = case.get('latest_message', {"text": case['input'], "intent": {"name": ""}})
        except:
            latest_message = {"text": case['input'], "intent": {"name": ""}}
        print(f"Latest message: {latest_message}")

        tracker = test.create_tracker(slots=case['slots'], latest_message=latest_message)
        
        try:
            dispatcher = CollectingDispatcher()


            
            result = await test.form_validator.validate_user_municipality_temp(
                case['input'],
                dispatcher,
                tracker,
                test.domain
            )
            

            print(f"Actual result: {result}")
            print(f"Dispatcher messages: {dispatcher.messages}")
            
            try:
                test.assertEqual(result, case['expected_result'])
                print(f"\n✅ Test PASSED: {case['name']}")
            except AssertionError as e:
                print(f"\n❌ Test FAILED: {case['name']}")
                print(f"Reason: {str(e)}")
                
        except Exception as e:
            print(f"\n❌ Test ERROR: {case['name']}")
            print(f"Error: {str(e)}")
            print(f"Type: {type(e).__name__}")

# Run municipality tests
await test_municipality_validation()


Testing: Valid municipality - Arjundhara

Input: Arjundhara Municipality
Expected result: {'user_municipality_temp': 'Arjundhara'}
Latest message: {'text': 'Arjundhara Municipality', 'intent': {'name': ''}}
######## FORM: Validating municipality_temp ######
Received value: Arjundhara Municipality
######## LocationValidator: Preprocessed text: arjundhara
######## LocationValidator: QR
######## LocationValidator: Score: 90.0
######## LocationValidator: Score: 100.0
######## LocationValidator: Municipality names: ['arjundhara', 'barhadashi', 'bhadrapur', 'birtamod', 'buddha shanti', 'damak', 'gauradaha', 'gaurigunj', 'haldibari', 'jhapa', 'kachanakawal', 'kamal', 'kankai', 'mechinagar', 'sitasatakshi']
######## LocationValidator: Score: 100.0
######## LocationValidator: Result: {'province': 'koshi province', 'district': 'jhapa', 'municipality': 'arjundhara'}
✅ Municipality validated: Arjundhara
Validated municipality: Arjundhara
Actual result: {'user_municipality_temp': 'Arjundhara'}
Dis

In [5]:
# Test all ValidateLocationForm functions
async def test_location_form_validations():
    """Test all validation functions in ValidateLocationForm."""
    test = TestValidateLocationForm()
    test.setUp()
    
    # Test cases organized by validation function
    test_cases = {
        "address_temp": [
            {
                "name": "Valid address",
                "input": "Ward 5, Near Temple",
                "slots": {
                    "requested_slot": "user_address_temp",
                    "user_municipality": "Arjundhara Municipality"
                },
                "expected_result": {"user_address_temp": "Ward 5, Near Temple"}
            },
            {
                "name": "Too short address",
                "input": "a",
                "slots": {
                    "requested_slot": "user_address_temp",
                    "user_municipality": "Arjundhara Municipality"
                },
                "expected_result": {"user_address_temp": None}
            },
            {
                "name": "Skip address",
                "input": "skip",
                "slots": {
                    "requested_slot": "user_address_temp",
                    "user_municipality": "Arjundhara Municipality"
                },
                "latest_message": {"text": "skip", "intent": {"name": "skip"}},
                "expected_result": {"user_address_temp": "Skipped"}
            }
        ],
        "address_confirmed": [
            {
                "name": "Confirm address",
                "input": True,
                "slots": {
                    "requested_slot": "user_address_confirmed",
                    "user_address_temp": "Ward 5, Near Temple",
                    "user_municipality": "Arjundhara Municipality"
                },
                "latest_message": {"text": "/affirm", "intent": {"name": "affirm"}},
                "expected_result": {"user_address": "Ward 5, Near Temple"}
            },
            {
                "name": "Reject address",
                "input": False,
                "slots": {
                    "requested_slot": "user_address_confirmed",
                    "user_address_temp": "Ward 5, Near Temple",
                    "user_municipality": "Arjundhara Municipality"
                },
                "latest_message": {"text": "/deny", "intent": {"name": "deny"}},
                "expected_result": {
                    "user_village": None,
                    "user_address": None,
                    "user_address_temp": None,
                    "user_address_confirmed": None
                }
            }
        ],
        "municipality_confirmed": [
            {
                "name": "Confirm municipality",
                "input": True,
                "slots": {
                    "requested_slot": "user_municipality_confirmed",
                    "user_municipality_temp": "Arjundhara Municipality"
                },
                "latest_message": {"text": "/affirm", "intent": {"name": "affirm"}},
                "expected_result": {
                    "user_municipality": "Arjundhara Municipality",
                    "user_municipality_confirmed": True
                }
            },
            {
                "name": "Reject municipality",
                "input": False,
                "slots": {
                    "requested_slot": "user_municipality_confirmed",
                    "user_municipality_temp": "Arjundhara Municipality"
                },
                "latest_message": {"text": "/deny", "intent": {"name": "deny"}},
                "expected_result": {
                    "user_municipality_temp": None,
                    "user_municipality": None,
                    "user_municipality_confirmed": None
                }
            }
        ]
    }
    
    # Test each validation function
    for func_name, cases in test_cases.items():
        print(f"\n{'='*20} Testing {func_name} validation {'='*20}")
        
        for case in cases:
            print(f"\n{'='*50}")
            print(f"Testing: {case['name']}")
            print(f"{'='*50}")
            
            try:
                dispatcher = CollectingDispatcher()
                latest_message = case.get('latest_message', {"text": case['input'], "intent": {"name": ""}})
                tracker = test.create_tracker(slots=case['slots'], latest_message=latest_message)
                
                # Call appropriate validation function
                if func_name == "address_temp":
                    result = await test.form_validator.validate_user_address_temp(
                        case['input'], dispatcher, tracker, test.domain
                    )
                elif func_name == "address_confirmed":
                    result = await test.form_validator.validate_user_address_confirmed(
                        case['input'], dispatcher, tracker, test.domain
                    )
                elif func_name == "municipality_confirmed":
                    result = await test.form_validator.validate_user_municipality_confirmed(
                        case['input'], dispatcher, tracker, test.domain
                    )
                
                print(f"\nInput: {case['input']}")
                print(f"Expected result: {case['expected_result']}")
                print(f"Actual result: {result}")
                print(f"Dispatcher messages: {dispatcher.messages}")
                
                try:
                    test.assertEqual(result, case['expected_result'])
                    print(f"\n✅ Test PASSED: {case['name']}")
                except AssertionError as e:
                    print(f"\n❌ Test FAILED: {case['name']}")
                    print(f"Reason: {str(e)}")
                    
            except Exception as e:
                print(f"\n❌ Test ERROR: {case['name']}")
                print(f"Error: {str(e)}")
                print(f"Type: {type(e).__name__}")

# Run all validation tests
await test_location_form_validations()


==================== Testing address_temp validation ====================

Testing: Valid address

Input: Ward 5, Near Temple
Expected result: {'user_address_temp': 'Ward 5, Near Temple'}
Actual result: {'user_address_temp': 'Ward 5, Near Temple'}
Dispatcher messages: [{'text': 'Thank you for providing your location details:\n            - Municipality: Arjundhara Municipality\n            - Village: None\n            - Address: None\n            Is this correct?', 'buttons': [{'title': 'Yes', 'payload': '/affirm'}, {'title': 'No', 'payload': '/deny'}], 'elements': [], 'custom': {}, 'template': None, 'response': None, 'image': None, 'attachment': None}]

✅ Test PASSED: Valid address

Testing: Too short address

Input: a
Expected result: {'user_address_temp': None}
Actual result: {'user_address_temp': None}
Dispatcher messages: [{'text': 'Please provide a valid address (at least 3 characters)', 'buttons': [], 'elements': [], 'custom': {}, 'template': None, 'response': None, 'image': No

In [6]:
# Municipality validation tests
async def test_municipality_validation():
    """Test municipality validation with different inputs."""
    test = TestValidateLocationForm()
    test.setUp()
    
    test_cases = [
        # Exact matches
        {
            "name": "Valid municipality - Arjundhara",
            "input": "Arjundhara Municipality",
            "slots": {"requested_slot": "user_municipality_temp"},
            "expected_result": {"user_municipality_temp": "Arjundhara"}
        },
        {
            "name": "Valid municipality - Kamal Rural Municipality",
            "input": "Kamal Rural Municipality",
            "slots": {"requested_slot": "user_municipality_temp"},
            "expected_result": {"user_municipality_temp": "Kamal"}
        },
        
        # Fuzzy matches - Single word
        {
            "name": "Fuzzy match - arjundhara lowercase",
            "input": "arjundhara",
            "slots": {"requested_slot": "user_municipality_temp"},
            "expected_result": {"user_municipality_temp": "Arjundhara"}
        },
        {
            "name": "Fuzzy match - arjundara typo",
            "input": "arjundara",
            "slots": {"requested_slot": "user_municipality_temp"},
            "expected_result": {"user_municipality_temp": "Arjundhara"}
        },
        
        # Fuzzy matches - Multi word
        {
            "name": "Fuzzy match - Kamal only",
            "input": "kamal",
            "slots": {"requested_slot": "user_municipality_temp"},
            "expected_result": {"user_municipality_temp": "Kamal"}
        },
        {
            "name": "Fuzzy match - Kamal Gaun",
            "input": "Kamal Gaun",
            "slots": {"requested_slot": "user_municipality_temp"},
            "expected_result": {"user_municipality_temp": "Kamal"}
        },
        {
            "name": "Fuzzy match - kamal gaun palika",
            "input": "kamal gaun palika",
            "slots": {"requested_slot": "user_municipality_temp"},
            "expected_result": {"user_municipality_temp": "Kamal"}
        },
        {
            "name": "Fuzzy match - kml gau",
            "input": "kml gau",
            "slots": {"requested_slot": "user_municipality_temp"},
            "expected_result": {"user_municipality_temp": "Kamal"}
        },
        
        # # Nepali script
        # {
        #     "name": "Nepali input - कमल",
        #     "input": "कमल",
        #     "slots": {"requested_slot": "user_municipality_temp"},
        #     "expected_result": {"user_municipality_temp": "Kamal Rural Municipality"}
        # },
        # {
        #     "name": "Nepali input - कमल गाउँपालिका",
        #     "input": "कमल गाउँपालिका",
        #     "slots": {"requested_slot": "user_municipality_temp"},
        #     "expected_result": {"user_municipality_temp": "Kamal Rural Municipality"}
        # },
        
        # Skip and Invalid cases
        {
            "name": "Skip request",
            "input": "skip",
            "slots": {"requested_slot": "user_municipality_temp"},
            "latest_message": {"text": "skip", "intent": {"name": "skip"}},
            "expected_result": {
                "user_municipality_temp": "Skipped",
                "user_municipality_confirmed": True,
                "provide_additional_location": False,
                "user_municipality": "Skipped"
            }
        },
        {
            "name": "Invalid municipality",
            "input": "NonExistent City",
            "slots": {"requested_slot": "user_municipality_temp"},
            "expected_result": {"user_municipality_temp": None}
        }
    ]
    
    for case in test_cases:
        print(f"\n{'='*50}")
        print(f"Testing: {case['name']}")
        print(f"{'='*50}")
        print(f"\nInput: {case['input']}")
        print(f"Expected result: {case['expected_result']}")
        
        try:
            latest_message = case.get('latest_message', {"text": case['input'], "intent": {"name": ""}})
        except:
            latest_message = {"text": case['input'], "intent": {"name": ""}}
        print(f"Latest message: {latest_message}")

        tracker = test.create_tracker(slots=case['slots'], latest_message=latest_message)
        
        try:
            dispatcher = CollectingDispatcher()


            
            result = await test.form_validator.validate_user_municipality_temp(
                case['input'],
                dispatcher,
                tracker,
                test.domain
            )
            

            print(f"Actual result: {result}")
            print(f"Dispatcher messages: {dispatcher.messages}")
            
            try:
                test.assertEqual(result, case['expected_result'])
                print(f"\n✅ Test PASSED: {case['name']}")
            except AssertionError as e:
                print(f"\n❌ Test FAILED: {case['name']}")
                print(f"Reason: {str(e)}")
                
        except Exception as e:
            print(f"\n❌ Test ERROR: {case['name']}")
            print(f"Error: {str(e)}")
            print(f"Type: {type(e).__name__}")

# Run municipality tests
await test_municipality_validation()


Testing: Valid municipality - Arjundhara

Input: Arjundhara Municipality
Expected result: {'user_municipality_temp': 'Arjundhara'}
Latest message: {'text': 'Arjundhara Municipality', 'intent': {'name': ''}}
######## FORM: Validating municipality_temp ######
Received value: Arjundhara Municipality
######## LocationValidator: Preprocessed text: arjundhara
######## LocationValidator: QR
######## LocationValidator: Score: 90.0
######## LocationValidator: Score: 100.0
######## LocationValidator: Municipality names: ['arjundhara', 'barhadashi', 'bhadrapur', 'birtamod', 'buddha shanti', 'damak', 'gauradaha', 'gaurigunj', 'haldibari', 'jhapa', 'kachanakawal', 'kamal', 'kankai', 'mechinagar', 'sitasatakshi']
######## LocationValidator: Score: 100.0
######## LocationValidator: Result: {'province': 'koshi province', 'district': 'jhapa', 'municipality': 'arjundhara'}
✅ Municipality validated: Arjundhara
Validated municipality: Arjundhara
Actual result: {'user_municipality_temp': 'Arjundhara'}
Dis

In [7]:
async def test_location_form_flows():
    """Test complete flows through the location form."""
    test = TestValidateLocationForm()
    test.setUp()
    
    flows = [
        {
            "name": "Happy Path - Complete Flow",
            "steps": [
                {
                    "name": "Enter municipality",
                    "function": "validate_user_municipality_temp",
                    "input": "Arjundhara",
                    "slots": {"requested_slot": "user_municipality_temp"},
                    "expected": {"user_municipality_temp": "Arjundhara"}
                },
                {
                    "name": "Confirm municipality",
                    "function": "validate_user_municipality_confirmed",
                    "input": True,
                    "slots": {
                        "requested_slot": "user_municipality_confirmed",
                        "user_municipality_temp": "Arjundhara"
                    },
                    "latest_message": {"text": "/affirm", "intent": {"name": "affirm"}},
                    "expected": {
                        "user_municipality": "Arjundhara",
                        "user_municipality_confirmed": True
                    }
                },
                {
                    "name": "Enter village",
                    "function": "validate_user_village",
                    "input": "Sanischare",
                    "slots": {
                        "requested_slot": "user_village",
                        "user_municipality": "Arjundhara"
                    },
                    "expected": {"user_village": "Sanischare"}
                },
                {
                    "name": "Enter address",
                    "function": "validate_user_address_temp",
                    "input": "Ward 5, Near Temple",
                    "slots": {
                        "requested_slot": "user_address_temp",
                        "user_municipality": "Arjundhara",
                        "user_village": "Sanischare"
                    },
                    "expected": {"user_address_temp": "Ward 5, Near Temple"}
                },
                {
                    "name": "Confirm address",
                    "function": "validate_user_address_confirmed",
                    "input": True,
                    "slots": {
                        "requested_slot": "user_address_confirmed",
                        "user_municipality": "Arjundhara",
                        "user_village": "Sanischare",
                        "user_address_temp": "Ward 5, Near Temple"
                    },
                    "latest_message": {"text": "/affirm", "intent": {"name": "affirm"}},
                    "expected": {"user_address": "Ward 5, Near Temple"}
                }
            ]
        },
        {
            "name": "Minimal Flow - Skip Additional Details",
            "steps": [
                {
                    "name": "Enter municipality",
                    "function": "validate_user_municipality_temp",
                    "input": "Kamal",
                    "slots": {"requested_slot": "user_municipality_temp"},
                    "expected": {"user_municipality_temp": "Kamal"}
                },
                {
                    "name": "Confirm municipality",
                    "function": "validate_user_municipality_confirmed",
                    "input": True,
                    "slots": {
                        "requested_slot": "user_municipality_confirmed",
                        "user_municipality_temp": "Kamal"
                    },
                    "latest_message": {"text": "/affirm", "intent": {"name": "affirm"}},
                    "expected": {
                        "user_municipality": "Kamal",
                        "user_municipality_confirmed": True
                    }
                },
                {
                    "name": "Skip additional details",
                    "function": "validate_user_village",
                    "input": "skip",
                    "slots": {
                        "requested_slot": "user_village",
                        "user_municipality": "Kamal"
                    },
                    "latest_message": {"text": "skip", "intent": {"name": "skip"}},
                    "expected": {"user_village": "Skipped"}
                }
            ]
        },
        {
            "name": "Correction Flow - Reject and Reenter Municipality",
            "steps": [
                {
                    "name": "Enter wrong municipality",
                    "function": "validate_user_municipality_temp",
                    "input": "Birtamod",
                    "slots": {"requested_slot": "user_municipality_temp"},
                    "expected": {"user_municipality_temp": "Birtamod"}
                },
                {
                    "name": "Reject municipality",
                    "function": "validate_user_municipality_confirmed",
                    "input": False,
                    "slots": {
                        "requested_slot": "user_municipality_confirmed",
                        "user_municipality_temp": "Birtamod"
                    },
                    "latest_message": {"text": "/deny", "intent": {"name": "deny"}},
                    "expected": {
                        "user_municipality_temp": None,
                        "user_municipality": None,
                        "user_municipality_confirmed": None
                    }
                },
                {
                    "name": "Enter correct municipality",
                    "function": "validate_user_municipality_temp",
                    "input": "Arjundhara",
                    "slots": {"requested_slot": "user_municipality_temp"},
                    "expected": {"user_municipality_temp": "Arjundhara"}
                }
            ]
        },
        {
            "name": "Fuzzy Match Flow",
            "steps": [
                {
                    "name": "Enter partial municipality name",
                    "function": "validate_user_municipality_temp",
                    "input": "kml",
                    "slots": {"requested_slot": "user_municipality_temp"},
                    "expected": {"user_municipality_temp": "Kamal"}
                },
                {
                    "name": "Confirm fuzzy matched municipality",
                    "function": "validate_user_municipality_confirmed",
                    "input": True,
                    "slots": {
                        "requested_slot": "user_municipality_confirmed",
                        "user_municipality_temp": "Kamal"
                    },
                    "latest_message": {"text": "/affirm", "intent": {"name": "affirm"}},
                    "expected": {
                        "user_municipality": "Kamal",
                        "user_municipality_confirmed": True
                    }
                }
            ]
        }
    ]
    
    for flow in flows:
        print(f"\n{'='*20} Testing Flow: {flow['name']} {'='*20}")
        accumulated_slots = {}
        
        for step in flow["steps"]:
            print(f"\n{'-'*50}")
            print(f"Step: {step['name']}")
            print(f"{'-'*50}")
            
            try:
                dispatcher = CollectingDispatcher()
                
                # Update slots with accumulated values
                current_slots = {**step['slots'], **accumulated_slots}
                latest_message = step.get('latest_message', {"text": step['input'], "intent": {"name": ""}})
                tracker = test.create_tracker(slots=current_slots, latest_message=latest_message)
                
                # Call appropriate validation function
                validation_function = getattr(test.form_validator, step['function'])
                result = await validation_function(step['input'], dispatcher, tracker, test.domain)
                
                print(f"\nInput: {step['input']}")
                print(f"Expected result: {step['expected']}")
                print(f"Actual result: {result}")
                print(f"Dispatcher messages: {dispatcher.messages}")
                
                try:
                    test.assertEqual(result, step['expected'])
                    print(f"\n✅ Step PASSED: {step['name']}")
                    # Update accumulated slots with the result
                    accumulated_slots.update(result)
                except AssertionError as e:
                    print(f"\n❌ Step FAILED: {step['name']}")
                    print(f"Reason: {str(e)}")
                    
            except Exception as e:
                print(f"\n❌ Step ERROR: {step['name']}")
                print(f"Error: {str(e)}")
                print(f"Type: {type(e).__name__}")

# Run flow tests
await test_location_form_flows()


==================== Testing Flow: Happy Path - Complete Flow ====================

--------------------------------------------------
Step: Enter municipality
--------------------------------------------------
######## FORM: Validating municipality_temp ######
Received value: Arjundhara
######## LocationValidator: Preprocessed text: arjundhara
######## LocationValidator: QR
######## LocationValidator: Score: 90.0
######## LocationValidator: Score: 100.0
######## LocationValidator: Municipality names: ['arjundhara', 'barhadashi', 'bhadrapur', 'birtamod', 'buddha shanti', 'damak', 'gauradaha', 'gaurigunj', 'haldibari', 'jhapa', 'kachanakawal', 'kamal', 'kankai', 'mechinagar', 'sitasatakshi']
######## LocationValidator: Score: 100.0
######## LocationValidator: Result: {'province': 'koshi province', 'district': 'jhapa', 'municipality': 'arjundhara'}
✅ Municipality validated: Arjundhara
Validated municipality: Arjundhara

Input: Arjundhara
Expected result: {'user_municipality_temp': 'Arju

In [8]:
# Municipality validation tests
async def test_municipality_validation():
    """Test municipality validation with different inputs."""
    test = TestValidateLocationForm()
    test.setUp()
    
    test_cases = [
        # Exact matches
        {
            "name": "Valid municipality - Arjundhara",
            "input": "Arjundhara Municipality",
            "slots": {"requested_slot": "user_municipality_temp"},
            "expected_result": {"user_municipality_temp": "Arjundhara"}
        },
        {
            "name": "Valid municipality - Kamal Rural Municipality",
            "input": "Kamal Rural Municipality",
            "slots": {"requested_slot": "user_municipality_temp"},
            "expected_result": {"user_municipality_temp": "Kamal"}
        },
        
        # Fuzzy matches - Single word
        {
            "name": "Fuzzy match - arjundhara lowercase",
            "input": "arjundhara",
            "slots": {"requested_slot": "user_municipality_temp"},
            "expected_result": {"user_municipality_temp": "Arjundhara"}
        },
        {
            "name": "Fuzzy match - arjundara typo",
            "input": "arjundara",
            "slots": {"requested_slot": "user_municipality_temp"},
            "expected_result": {"user_municipality_temp": "Arjundhara"}
        },
        
        # Fuzzy matches - Multi word
        {
            "name": "Fuzzy match - Kamal only",
            "input": "kamal",
            "slots": {"requested_slot": "user_municipality_temp"},
            "expected_result": {"user_municipality_temp": "Kamal"}
        },
        {
            "name": "Fuzzy match - Kamal Gaun",
            "input": "Kamal Gaun",
            "slots": {"requested_slot": "user_municipality_temp"},
            "expected_result": {"user_municipality_temp": "Kamal"}
        },
        {
            "name": "Fuzzy match - kamal gaun palika",
            "input": "kamal gaun palika",
            "slots": {"requested_slot": "user_municipality_temp"},
            "expected_result": {"user_municipality_temp": "Kamal"}
        },
        {
            "name": "Fuzzy match - kml gau",
            "input": "kml gau",
            "slots": {"requested_slot": "user_municipality_temp"},
            "expected_result": {"user_municipality_temp": "Kamal"}
        },
        
        # # Nepali script
        # {
        #     "name": "Nepali input - कमल",
        #     "input": "कमल",
        #     "slots": {"requested_slot": "user_municipality_temp"},
        #     "expected_result": {"user_municipality_temp": "Kamal Rural Municipality"}
        # },
        # {
        #     "name": "Nepali input - कमल गाउँपालिका",
        #     "input": "कमल गाउँपालिका",
        #     "slots": {"requested_slot": "user_municipality_temp"},
        #     "expected_result": {"user_municipality_temp": "Kamal Rural Municipality"}
        # },
        
        # Skip and Invalid cases
        {
            "name": "Skip request",
            "input": "skip",
            "slots": {"requested_slot": "user_municipality_temp"},
            "latest_message": {"text": "skip", "intent": {"name": "skip"}},
            "expected_result": {
                "user_municipality_temp": "Skipped",
                "user_municipality_confirmed": True,
                "provide_additional_location": False,
                "user_municipality": "Skipped"
            }
        },
        {
            "name": "Invalid municipality",
            "input": "NonExistent City",
            "slots": {"requested_slot": "user_municipality_temp"},
            "expected_result": {"user_municipality_temp": None}
        }
    ]
    
    for case in test_cases:
        print(f"\n{'='*50}")
        print(f"Testing: {case['name']}")
        print(f"{'='*50}")
        print(f"\nInput: {case['input']}")
        print(f"Expected result: {case['expected_result']}")
        
        try:
            latest_message = case.get('latest_message', {"text": case['input'], "intent": {"name": ""}})
        except:
            latest_message = {"text": case['input'], "intent": {"name": ""}}
        print(f"Latest message: {latest_message}")

        tracker = test.create_tracker(slots=case['slots'], latest_message=latest_message)
        
        try:
            dispatcher = CollectingDispatcher()


            
            result = await test.form_validator.validate_user_municipality_temp(
                case['input'],
                dispatcher,
                tracker,
                test.domain
            )
            

            print(f"Actual result: {result}")
            print(f"Dispatcher messages: {dispatcher.messages}")
            
            try:
                test.assertEqual(result, case['expected_result'])
                print(f"\n✅ Test PASSED: {case['name']}")
            except AssertionError as e:
                print(f"\n❌ Test FAILED: {case['name']}")
                print(f"Reason: {str(e)}")
                
        except Exception as e:
            print(f"\n❌ Test ERROR: {case['name']}")
            print(f"Error: {str(e)}")
            print(f"Type: {type(e).__name__}")

# Run municipality tests
await test_municipality_validation()


Testing: Valid municipality - Arjundhara

Input: Arjundhara Municipality
Expected result: {'user_municipality_temp': 'Arjundhara'}
Latest message: {'text': 'Arjundhara Municipality', 'intent': {'name': ''}}
######## FORM: Validating municipality_temp ######
Received value: Arjundhara Municipality
######## LocationValidator: Preprocessed text: arjundhara
######## LocationValidator: QR
######## LocationValidator: Score: 90.0
######## LocationValidator: Score: 100.0
######## LocationValidator: Municipality names: ['arjundhara', 'barhadashi', 'bhadrapur', 'birtamod', 'buddha shanti', 'damak', 'gauradaha', 'gaurigunj', 'haldibari', 'jhapa', 'kachanakawal', 'kamal', 'kankai', 'mechinagar', 'sitasatakshi']
######## LocationValidator: Score: 100.0
######## LocationValidator: Result: {'province': 'koshi province', 'district': 'jhapa', 'municipality': 'arjundhara'}
✅ Municipality validated: Arjundhara
Validated municipality: Arjundhara
Actual result: {'user_municipality_temp': 'Arjundhara'}
Dis

In [9]:
# Municipality validation tests
async def test_municipality_validation():
    """Test municipality validation with different inputs."""
    test = TestValidateLocationForm()
    test.setUp()
    
    test_cases = [
        # Exact matches
        {
            "name": "Valid municipality - Arjundhara",
            "input": "Arjundhara Municipality",
            "slots": {"requested_slot": "user_municipality_temp"},
            "expected_result": {"user_municipality_temp": "Arjundhara"}
        },
        {
            "name": "Valid municipality - Kamal Rural Municipality",
            "input": "Kamal Rural Municipality",
            "slots": {"requested_slot": "user_municipality_temp"},
            "expected_result": {"user_municipality_temp": "Kamal"}
        },
        
        # Fuzzy matches - Single word
        {
            "name": "Fuzzy match - arjundhara lowercase",
            "input": "arjundhara",
            "slots": {"requested_slot": "user_municipality_temp"},
            "expected_result": {"user_municipality_temp": "Arjundhara"}
        },
        {
            "name": "Fuzzy match - arjundara typo",
            "input": "arjundara",
            "slots": {"requested_slot": "user_municipality_temp"},
            "expected_result": {"user_municipality_temp": "Arjundhara"}
        },
        
        # Fuzzy matches - Multi word
        {
            "name": "Fuzzy match - Kamal only",
            "input": "kamal",
            "slots": {"requested_slot": "user_municipality_temp"},
            "expected_result": {"user_municipality_temp": "Kamal"}
        },
        {
            "name": "Fuzzy match - Kamal Gaun",
            "input": "Kamal Gaun",
            "slots": {"requested_slot": "user_municipality_temp"},
            "expected_result": {"user_municipality_temp": "Kamal"}
        },
        {
            "name": "Fuzzy match - kamal gaun palika",
            "input": "kamal gaun palika",
            "slots": {"requested_slot": "user_municipality_temp"},
            "expected_result": {"user_municipality_temp": "Kamal"}
        },
        {
            "name": "Fuzzy match - kml gau",
            "input": "kml gau",
            "slots": {"requested_slot": "user_municipality_temp"},
            "expected_result": {"user_municipality_temp": "Kamal"}
        },
        
        # # Nepali script
        # {
        #     "name": "Nepali input - कमल",
        #     "input": "कमल",
        #     "slots": {"requested_slot": "user_municipality_temp"},
        #     "expected_result": {"user_municipality_temp": "Kamal Rural Municipality"}
        # },
        # {
        #     "name": "Nepali input - कमल गाउँपालिका",
        #     "input": "कमल गाउँपालिका",
        #     "slots": {"requested_slot": "user_municipality_temp"},
        #     "expected_result": {"user_municipality_temp": "Kamal Rural Municipality"}
        # },
        
        # Skip and Invalid cases
        {
            "name": "Skip request",
            "input": "skip",
            "slots": {"requested_slot": "user_municipality_temp"},
            "latest_message": {"text": "skip", "intent": {"name": "skip"}},
            "expected_result": {
                "user_municipality_temp": "Skipped",
                "user_municipality_confirmed": True,
                "provide_additional_location": False,
                "user_municipality": "Skipped"
            }
        },
        {
            "name": "Invalid municipality",
            "input": "NonExistent City",
            "slots": {"requested_slot": "user_municipality_temp"},
            "expected_result": {"user_municipality_temp": None}
        }
    ]
    
    for case in test_cases:
        print(f"\n{'='*50}")
        print(f"Testing: {case['name']}")
        print(f"{'='*50}")
        print(f"\nInput: {case['input']}")
        print(f"Expected result: {case['expected_result']}")
        
        try:
            latest_message = case.get('latest_message', {"text": case['input'], "intent": {"name": ""}})
        except:
            latest_message = {"text": case['input'], "intent": {"name": ""}}
        print(f"Latest message: {latest_message}")

        tracker = test.create_tracker(slots=case['slots'], latest_message=latest_message)
        
        try:
            dispatcher = CollectingDispatcher()


            
            result = await test.form_validator.validate_user_municipality_temp(
                case['input'],
                dispatcher,
                tracker,
                test.domain
            )
            

            print(f"Actual result: {result}")
            print(f"Dispatcher messages: {dispatcher.messages}")
            
            try:
                test.assertEqual(result, case['expected_result'])
                print(f"\n✅ Test PASSED: {case['name']}")
            except AssertionError as e:
                print(f"\n❌ Test FAILED: {case['name']}")
                print(f"Reason: {str(e)}")
                
        except Exception as e:
            print(f"\n❌ Test ERROR: {case['name']}")
            print(f"Error: {str(e)}")
            print(f"Type: {type(e).__name__}")

# Run municipality tests
await test_municipality_validation()


Testing: Valid municipality - Arjundhara

Input: Arjundhara Municipality
Expected result: {'user_municipality_temp': 'Arjundhara'}
Latest message: {'text': 'Arjundhara Municipality', 'intent': {'name': ''}}
######## FORM: Validating municipality_temp ######
Received value: Arjundhara Municipality
######## LocationValidator: Preprocessed text: arjundhara
######## LocationValidator: QR
######## LocationValidator: Score: 90.0
######## LocationValidator: Score: 100.0
######## LocationValidator: Municipality names: ['arjundhara', 'barhadashi', 'bhadrapur', 'birtamod', 'buddha shanti', 'damak', 'gauradaha', 'gaurigunj', 'haldibari', 'jhapa', 'kachanakawal', 'kamal', 'kankai', 'mechinagar', 'sitasatakshi']
######## LocationValidator: Score: 100.0
######## LocationValidator: Result: {'province': 'koshi province', 'district': 'jhapa', 'municipality': 'arjundhara'}
✅ Municipality validated: Arjundhara
Validated municipality: Arjundhara
Actual result: {'user_municipality_temp': 'Arjundhara'}
Dis

In [10]:
async def test_location_form_flows():
    """Test complete flows through the location form."""
    test = TestValidateLocationForm()
    test.setUp()
    
    flows = [
        {
            "name": "Happy Path - Complete Flow",
            "steps": [
                {
                    "name": "Enter municipality",
                    "function": "validate_user_municipality_temp",
                    "input": "Arjundhara",
                    "slots": {"requested_slot": "user_municipality_temp"},
                    "expected": {"user_municipality_temp": "Arjundhara"}
                },
                {
                    "name": "Confirm municipality",
                    "function": "validate_user_municipality_confirmed",
                    "input": True,
                    "slots": {
                        "requested_slot": "user_municipality_confirmed",
                        "user_municipality_temp": "Arjundhara"
                    },
                    "latest_message": {"text": "/affirm", "intent": {"name": "affirm"}},
                    "expected": {
                        "user_municipality": "Arjundhara",
                        "user_municipality_confirmed": True
                    }
                },
                {
                    "name": "Enter village",
                    "function": "validate_user_village",
                    "input": "Sanischare",
                    "slots": {
                        "requested_slot": "user_village",
                        "user_municipality": "Arjundhara"
                    },
                    "expected": {"user_village": "Sanischare"}
                },
                {
                    "name": "Enter address",
                    "function": "validate_user_address_temp",
                    "input": "Ward 5, Near Temple",
                    "slots": {
                        "requested_slot": "user_address_temp",
                        "user_municipality": "Arjundhara",
                        "user_village": "Sanischare"
                    },
                    "expected": {"user_address_temp": "Ward 5, Near Temple"}
                },
                {
                    "name": "Confirm address",
                    "function": "validate_user_address_confirmed",
                    "input": True,
                    "slots": {
                        "requested_slot": "user_address_confirmed",
                        "user_municipality": "Arjundhara",
                        "user_village": "Sanischare",
                        "user_address_temp": "Ward 5, Near Temple"
                    },
                    "latest_message": {"text": "/affirm", "intent": {"name": "affirm"}},
                    "expected": {"user_address": "Ward 5, Near Temple"}
                }
            ]
        },
        {
            "name": "Minimal Flow - Skip Additional Details",
            "steps": [
                {
                    "name": "Enter municipality",
                    "function": "validate_user_municipality_temp",
                    "input": "Kamal",
                    "slots": {"requested_slot": "user_municipality_temp"},
                    "expected": {"user_municipality_temp": "Kamal"}
                },
                {
                    "name": "Confirm municipality",
                    "function": "validate_user_municipality_confirmed",
                    "input": True,
                    "slots": {
                        "requested_slot": "user_municipality_confirmed",
                        "user_municipality_temp": "Kamal"
                    },
                    "latest_message": {"text": "/affirm", "intent": {"name": "affirm"}},
                    "expected": {
                        "user_municipality": "Kamal",
                        "user_municipality_confirmed": True
                    }
                },
                {
                    "name": "Skip additional details",
                    "function": "validate_user_village",
                    "input": "skip",
                    "slots": {
                        "requested_slot": "user_village",
                        "user_municipality": "Kamal"
                    },
                    "latest_message": {"text": "skip", "intent": {"name": "skip"}},
                    "expected": {"user_village": "Skipped"}
                }
            ]
        },
        {
            "name": "Correction Flow - Reject and Reenter Municipality",
            "steps": [
                {
                    "name": "Enter wrong municipality",
                    "function": "validate_user_municipality_temp",
                    "input": "Birtamod",
                    "slots": {"requested_slot": "user_municipality_temp"},
                    "expected": {"user_municipality_temp": "Birtamod"}
                },
                {
                    "name": "Reject municipality",
                    "function": "validate_user_municipality_confirmed",
                    "input": False,
                    "slots": {
                        "requested_slot": "user_municipality_confirmed",
                        "user_municipality_temp": "Birtamod"
                    },
                    "latest_message": {"text": "/deny", "intent": {"name": "deny"}},
                    "expected": {
                        "user_municipality_temp": None,
                        "user_municipality": None,
                        "user_municipality_confirmed": None
                    }
                },
                {
                    "name": "Enter correct municipality",
                    "function": "validate_user_municipality_temp",
                    "input": "Arjundhara",
                    "slots": {"requested_slot": "user_municipality_temp"},
                    "expected": {"user_municipality_temp": "Arjundhara"}
                }
            ]
        },
        {
            "name": "Fuzzy Match Flow",
            "steps": [
                {
                    "name": "Enter partial municipality name",
                    "function": "validate_user_municipality_temp",
                    "input": "kml",
                    "slots": {"requested_slot": "user_municipality_temp"},
                    "expected": {"user_municipality_temp": "Kamal"}
                },
                {
                    "name": "Confirm fuzzy matched municipality",
                    "function": "validate_user_municipality_confirmed",
                    "input": True,
                    "slots": {
                        "requested_slot": "user_municipality_confirmed",
                        "user_municipality_temp": "Kamal"
                    },
                    "latest_message": {"text": "/affirm", "intent": {"name": "affirm"}},
                    "expected": {
                        "user_municipality": "Kamal",
                        "user_municipality_confirmed": True
                    }
                }
            ]
        },
        {
            "name": "Nepali Script Flow",
            "steps": [
                {
                    "name": "Enter municipality in Nepali",
                    "function": "validate_user_municipality_temp",
                    "input": "अर्जुनधारा",
                    "slots": {"requested_slot": "user_municipality_temp"},
                    "expected": {"user_municipality_temp": "Arjundhara"}
                },
                {
                    "name": "Confirm Nepali input",
                    "function": "validate_user_municipality_confirmed",
                    "input": True,
                    "slots": {
                        "requested_slot": "user_municipality_confirmed",
                        "user_municipality_temp": "Arjundhara"
                    },
                    "latest_message": {"text": "/affirm", "intent": {"name": "affirm"}},
                    "expected": {
                        "user_municipality": "Arjundhara",
                        "user_municipality_confirmed": True
                    }
                }
            ]
        },
        {
            "name": "Multiple Rejections Flow",
            "steps": [
                {
                    "name": "Enter first wrong municipality",
                    "function": "validate_user_municipality_temp",
                    "input": "Birtamod",
                    "slots": {"requested_slot": "user_municipality_temp"},
                    "expected": {"user_municipality_temp": "Birtamod"}
                },
                {
                    "name": "Reject first municipality",
                    "function": "validate_user_municipality_confirmed",
                    "input": False,
                    "slots": {
                        "requested_slot": "user_municipality_confirmed",
                        "user_municipality_temp": "Birtamod"
                    },
                    "latest_message": {"text": "/deny", "intent": {"name": "deny"}},
                    "expected": {
                        "user_municipality_temp": None,
                        "user_municipality": None,
                        "user_municipality_confirmed": None
                    }
                },
                {
                    "name": "Enter second wrong municipality",
                    "function": "validate_user_municipality_temp",
                    "input": "Damak",
                    "slots": {"requested_slot": "user_municipality_temp"},
                    "expected": {"user_municipality_temp": "Damak"}
                },
                {
                    "name": "Reject second municipality",
                    "function": "validate_user_municipality_confirmed",
                    "input": False,
                    "slots": {
                        "requested_slot": "user_municipality_confirmed",
                        "user_municipality_temp": "Damak"
                    },
                    "latest_message": {"text": "/deny", "intent": {"name": "deny"}},
                    "expected": {
                        "user_municipality_temp": None,
                        "user_municipality": None,
                        "user_municipality_confirmed": None
                    }
                }
            ]
        },
        {
            "name": "Address Correction Flow",
            "steps": [
                {
                    "name": "Enter municipality",
                    "function": "validate_user_municipality_temp",
                    "input": "Arjundhara",
                    "slots": {"requested_slot": "user_municipality_temp"},
                    "expected": {"user_municipality_temp": "Arjundhara"}
                },
                {
                    "name": "Confirm municipality",
                    "function": "validate_user_municipality_confirmed",
                    "input": True,
                    "slots": {
                        "requested_slot": "user_municipality_confirmed",
                        "user_municipality_temp": "Arjundhara"
                    },
                    "latest_message": {"text": "/affirm", "intent": {"name": "affirm"}},
                    "expected": {
                        "user_municipality": "Arjundhara",
                        "user_municipality_confirmed": True
                    }
                },
                {
                    "name": "Enter wrong address",
                    "function": "validate_user_address_temp",
                    "input": "Ward 5",
                    "slots": {
                        "requested_slot": "user_address_temp",
                        "user_municipality": "Arjundhara"
                    },
                    "expected": {"user_address_temp": "Ward 5"}
                },
                {
                    "name": "Reject address",
                    "function": "validate_user_address_confirmed",
                    "input": False,
                    "slots": {
                        "requested_slot": "user_address_confirmed",
                        "user_municipality": "Arjundhara",
                        "user_address_temp": "Ward 5"
                    },
                    "latest_message": {"text": "/deny", "intent": {"name": "deny"}},
                    "expected": {
                        "user_village": None,
                        "user_address": None,
                        "user_address_temp": None,
                        "user_address_confirmed": None
                    }
                }
            ]
        },
        {
            "name": "Edge Cases Flow",
            "steps": [
                {
                    "name": "Enter very short input",
                    "function": "validate_user_municipality_temp",
                    "input": "k",
                    "slots": {"requested_slot": "user_municipality_temp"},
                    "expected": {"user_municipality_temp": None}
                },
                {
                    "name": "Enter with extra spaces",
                    "function": "validate_user_municipality_temp",
                    "input": "  Arjundhara    ",
                    "slots": {"requested_slot": "user_municipality_temp"},
                    "expected": {"user_municipality_temp": "Arjundhara"}
                },
                {
                    "name": "Enter mixed case",
                    "function": "validate_user_municipality_temp",
                    "input": "aRjUnDhArA",
                    "slots": {"requested_slot": "user_municipality_temp"},
                    "expected": {"user_municipality_temp": "Arjundhara"}
                }
            ]
        },
{
            "name": "Skip Flow",
            "steps": [
                {
                    "name": "Skip municipality",
                    "function": "validate_user_municipality_temp",
                    "input": "skip",
                    "slots": {"requested_slot": "user_municipality_temp"},
                    "latest_message": {"text": "skip", "intent": {"name": "skip"}},
                    "expected": {
                        "user_municipality_temp": "slot_skipped",
                        "user_municipality_confirmed": True,
                        "provide_additional_location": False,
                        "user_municipality": "slot_skipped"
                    }
                }
            ]
        },
        {
            "name": "Skip Additional Details Flow",
            "steps": [
                {
                    "name": "Enter municipality",
                    "function": "validate_user_municipality_temp",
                    "input": "Arjundhara",
                    "slots": {"requested_slot": "user_municipality_temp"},
                    "expected": {"user_municipality_temp": "Arjundhara"}
                },
                {
                    "name": "Skip village",
                    "function": "validate_user_village",
                    "input": "skip",
                    "slots": {
                        "requested_slot": "user_village",
                        "user_municipality": "Arjundhara"
                    },
                    "latest_message": {"text": "skip", "intent": {"name": "skip"}},
                    "expected": {"user_village": "slot_skipped"}
                },
                {
                    "name": "Skip address",
                    "function": "validate_user_address_temp",
                    "input": "skip",
                    "slots": {
                        "requested_slot": "user_address_temp",
                        "user_municipality": "Arjundhara",
                        "user_village": "slot_skipped"
                    },
                    "latest_message": {"text": "skip", "intent": {"name": "skip"}},
                    "expected": {"user_address_temp": "slot_skipped"}
                }
            ]
        }
    ]

    
    
    
    for flow in flows:
        print(f"\n{'='*20} Testing Flow: {flow['name']} {'='*20}")
        accumulated_slots = {}
        
        for step in flow["steps"]:
            print(f"\n{'-'*50}")
            print(f"Step: {step['name']}")
            print(f"{'-'*50}")
            
            try:
                dispatcher = CollectingDispatcher()
                
                # Update slots with accumulated values
                current_slots = {**step['slots'], **accumulated_slots}
                latest_message = step.get('latest_message', {"text": step['input'], "intent": {"name": ""}})
                tracker = test.create_tracker(slots=current_slots, latest_message=latest_message)
                
                # Call appropriate validation function
                validation_function = getattr(test.form_validator, step['function'])
                result = await validation_function(step['input'], dispatcher, tracker, test.domain)
                
                print(f"\nInput: {step['input']}")
                print(f"Expected result: {step['expected']}")
                print(f"Actual result: {result}")
                print(f"Dispatcher messages: {dispatcher.messages}")
                
                try:
                    test.assertEqual(result, step['expected'])
                    print(f"\n✅ Step PASSED: {step['name']}")
                    # Update accumulated slots with the result
                    accumulated_slots.update(result)
                except AssertionError as e:
                    print(f"\n❌ Step FAILED: {step['name']}")
                    print(f"Reason: {str(e)}")
                    
            except Exception as e:
                print(f"\n❌ Step ERROR: {step['name']}")
                print(f"Error: {str(e)}")
                print(f"Type: {type(e).__name__}")

# Run flow tests
await test_location_form_flows()


==================== Testing Flow: Happy Path - Complete Flow ====================

--------------------------------------------------
Step: Enter municipality
--------------------------------------------------
######## FORM: Validating municipality_temp ######
Received value: Arjundhara
######## LocationValidator: Preprocessed text: arjundhara
######## LocationValidator: QR
######## LocationValidator: Score: 90.0
######## LocationValidator: Score: 100.0
######## LocationValidator: Municipality names: ['arjundhara', 'barhadashi', 'bhadrapur', 'birtamod', 'buddha shanti', 'damak', 'gauradaha', 'gaurigunj', 'haldibari', 'jhapa', 'kachanakawal', 'kamal', 'kankai', 'mechinagar', 'sitasatakshi']
######## LocationValidator: Score: 100.0
######## LocationValidator: Result: {'province': 'koshi province', 'district': 'jhapa', 'municipality': 'arjundhara'}
✅ Municipality validated: Arjundhara
Validated municipality: Arjundhara

Input: Arjundhara
Expected result: {'user_municipality_temp': 'Arju

In [11]:
# Define test flows
LOCATION_FORM_TEST_FLOWS = [
    {
        "name": "Skip Flow",
        "steps": [
            {
                "name": "Skip municipality",
                "function": "validate_user_municipality_temp",
                "input": "skip",
                "slots": {"requested_slot": "user_municipality_temp"},
                "latest_message": {"text": "skip", "intent": {"name": "skip"}},
                "expected": {
                    "user_municipality_temp": "slot_skipped",
                    "user_municipality_confirmed": True,
                    "provide_additional_location": False,
                    "user_municipality": "slot_skipped"
                }
            }
        ]
    },
    {
        "name": "Skip Additional Details Flow",
        "steps": [
            {
                "name": "Enter municipality",
                "function": "validate_user_municipality_temp",
                "input": "Arjundhara",
                "slots": {"requested_slot": "user_municipality_temp"},
                "expected": {"user_municipality_temp": "Arjundhara"}
            },
            {
                "name": "Skip village",
                "function": "validate_user_village",
                "input": "skip",
                "slots": {
                    "requested_slot": "user_village",
                    "user_municipality": "Arjundhara"
                },
                "latest_message": {"text": "skip", "intent": {"name": "skip"}},
                "expected": {"user_village": "slot_skipped"}
            },
            {
                "name": "Skip address",
                "function": "validate_user_address_temp",
                "input": "skip",
                "slots": {
                    "requested_slot": "user_address_temp",
                    "user_municipality": "Arjundhara",
                    "user_village": "slot_skipped"
                },
                "latest_message": {"text": "skip", "intent": {"name": "skip"}},
                "expected": {"user_address_temp": "slot_skipped"}
            }
        ]
    },
    {
        "name": "Skip Confirmation Flow",
        "steps": [
            {
                "name": "Detect potential skip word",
                "function": "validate_user_municipality_temp",
                "input": "pass",
                "slots": {"requested_slot": "user_municipality_temp"},
                "latest_message": {"text": "pass", "intent": {"name": ""}},
                "expected": {
                    "skip_validation_needed": "user_municipality_temp",
                    "skipped_detected_text": "pass"
                }
            },
            {
                "name": "Confirm skip",
                "function": "validate_user_municipality_temp",
                "input": "/affirm_skip",
                "slots": {
                    "requested_slot": "user_municipality_temp",
                    "skip_validation_needed": "user_municipality_temp",
                    "skipped_detected_text": "pass"
                },
                "latest_message": {"text": "/affirm_skip", "intent": {"name": "affirm_skip"}},
                "expected": {
                    "user_municipality_temp": "slot_skipped",
                    "skip_validation_needed": None,
                    "skipped_detected_text": None
                }
            }
        ]
    },
    {
        "name": "Deny Skip Flow",
        "steps": [
            {
                "name": "Detect potential skip word",
                "function": "validate_user_municipality_temp",
                "input": "pass",
                "slots": {"requested_slot": "user_municipality_temp"},
                "latest_message": {"text": "pass", "intent": {"name": ""}},
                "expected": {
                    "skip_validation_needed": "user_municipality_temp",
                    "skipped_detected_text": "pass"
                }
            },
            {
                "name": "Deny skip",
                "function": "validate_user_municipality_temp",
                "input": "/deny_skip",
                "slots": {
                    "requested_slot": "user_municipality_temp",
                    "skip_validation_needed": "user_municipality_temp",
                    "skipped_detected_text": "pass"
                },
                "latest_message": {"text": "/deny_skip", "intent": {"name": "deny_skip"}},
                "expected": {
                    "user_municipality_temp": "pass",
                    "skip_validation_needed": None,
                    "skipped_detected_text": None
                }
            }
        ]
    }
]



In [12]:

# Example of updated test flows
LOCATION_FORM_TEST_FLOWS = [
    {
        "name": "Normal Municipality Flow",
        "steps": [
            {
                "name": "Enter municipality temp",
                "function": "validate_user_municipality_temp",
                "input": "Arjundhara",
                "slots": {"requested_slot": "user_municipality_temp"},
                "latest_message": {"text": "Arjundhara", "intent": {"name": ""}},
                "expected": {"user_municipality_temp": "Arjundhara"}
            }
        ]
    },
    {
        "name": "Skip Flow",
        "steps": [
            {
                "name": "Skip municipality temp",
                "function": "validate_user_municipality_temp",
                "input": "skip",
                "slots": {"requested_slot": "user_municipality_temp"},
                "latest_message": {"text": "skip", "intent": {"name": "skip"}},
                "expected": {
                    "user_municipality_temp": "slot_skipped",
                    "user_municipality_confirmed": True,
                    "provide_additional_location": False,
                    "user_municipality": "slot_skipped"
                }
            }
        ]
    },
    {
        "name": "Complete Flow",
        "steps": [
            {
                "name": "Enter municipality temp",
                "function": "validate_user_municipality_temp",
                "input": "Arjundhara",
                "slots": {"requested_slot": "user_municipality_temp"},
                "expected": {"user_municipality_temp": "Arjundhara"}
            },
            {
                "name": "Enter village",
                "function": "validate_user_village",
                "input": "Ward 5",
                "slots": {
                    "requested_slot": "user_village",
                    "user_municipality_temp": "Arjundhara"
                },
                "expected": {"user_village": "Ward 5"}
            },
            {
                "name": "Enter address temp",
                "function": "validate_user_address_temp",
                "input": "Near Temple",
                "slots": {
                    "requested_slot": "user_address_temp",
                    "user_municipality_temp": "Arjundhara",
                    "user_village": "Ward 5"
                },
                "expected": {"user_address_temp": "Near Temple"}
            }
        ]
    }
]


In [13]:
async def test_location_form_flows(flows):
    """Test location form flows."""
    test = TestValidateLocationForm()
    test.setUp()
    
    for flow in flows:
        print(f"\n{'='*20} Testing Flow: {flow['name']} {'='*20}")
        accumulated_slots = {}
        
        for step in flow["steps"]:
            print(f"\n{'-'*50}")
            print(f"Step: {step['name']}")
            print(f"{'-'*50}")
            
            try:
                dispatcher = CollectingDispatcher()
                current_slots = {**step['slots'], **accumulated_slots}
                latest_message = step.get('latest_message', {"text": step['input'], "intent": {"name": ""}})
                tracker = test.create_tracker(slots=current_slots, latest_message=latest_message)
                
                # Mock _is_skip_requested
                with patch.object(
                    test.form_validator, 
                    '_is_skip_requested',
                    return_value=(
                        latest_message.get('intent', {}).get('name') == 'skip',
                        False,
                        None
                    )
                ):
                    slot_name = step['function'].replace('validate_', '')
                    slot_value = step['input']  # Default to input value
                    print(f"Slot name: {slot_name}")
                    print(f"Slot value: {slot_value}")
                    
                    # Skip extraction for user_municipality and user_address
                    if not slot_name in ['user_municipality', 'user_address']:
                        extraction_function = getattr(test.form_validator, f'extract_{slot_name}')
                        extraction_result = await extraction_function(
                            dispatcher,
                            tracker,
                            test.domain
                        )
                        print(f"\nExtraction result: {extraction_result}")
                        
                        # Update slot_value with extraction result if available
                        if isinstance(extraction_result, dict):
                            current_slots.update(extraction_result)
                            tracker = test.create_tracker(slots=current_slots, latest_message=latest_message)
                            # Use extracted value for validation if available
                            slot_value = extraction_result.get(slot_name, slot_value)
                    
                    # Call validation with the extracted value
                    validation_function = getattr(test.form_validator, f'validate_{slot_name}')
                    validation_result = await validation_function(
                        slot_value,  # Use extracted value here
                        dispatcher,
                        tracker,
                        test.domain
                    )
                    
                    print(f"\nInput: {step['input']}")
                    print(f"Slot value for validation: {slot_value}")
                    print(f"Expected result: {step['expected']}")
                    print(f"Validation result: {validation_result}")
                    print(f"Dispatcher messages: {dispatcher.messages}")
                    
                    try:
                        test.assertEqual(validation_result, step['expected'])
                        print(f"\n✅ Step PASSED: {step['name']}")
                        accumulated_slots.update(validation_result)
                    except AssertionError as e:
                        print(f"\n❌ Step FAILED: {step['name']}")
                        print(f"Expected: {step['expected']}")
                        print(f"Got: {validation_result}")
                        print(f"Difference: {str(e)}")
                    
            except Exception as e:
                print(f"\n❌ Step ERROR: {step['name']}")
                print(f"Error: {str(e)}")
                print(f"Type: {type(e).__name__}")
                import traceback
                print(f"Traceback: {traceback.format_exc()}")

# The test flows remain the same
await test_location_form_flows(LOCATION_FORM_TEST_FLOWS)


==================== Testing Flow: Normal Municipality Flow ====================

--------------------------------------------------
Step: Enter municipality temp
--------------------------------------------------
Slot name: user_municipality_temp
Slot value: Arjundhara

Extraction result: {'user_municipality_temp': 'Arjundhara'}
######## FORM: Validating municipality_temp ######
Received value: Arjundhara
######## LocationValidator: Preprocessed text: arjundhara
######## LocationValidator: QR
######## LocationValidator: Score: 90.0
######## LocationValidator: Score: 100.0
######## LocationValidator: Municipality names: ['arjundhara', 'barhadashi', 'bhadrapur', 'birtamod', 'buddha shanti', 'damak', 'gauradaha', 'gaurigunj', 'haldibari', 'jhapa', 'kachanakawal', 'kamal', 'kankai', 'mechinagar', 'sitasatakshi']
######## LocationValidator: Score: 100.0
######## LocationValidator: Result: {'province': 'koshi province', 'district': 'jhapa', 'municipality': 'arjundhara'}
✅ Municipality vali